In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [2]:
# URL for database
#URL = 'https://en.wikipedia.org/wiki/List_of_birds_of_the_Netherlands'
#URL = 'https://en.wikipedia.org/wiki/List_of_mammals_of_the_Netherlands'
URL = 'https://en.wikipedia.org/wiki/List_of_mammals_of_Europe'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

# Find all wikiparts
Animals = soup.find_all('a')
# Create links 
AnimalsWikiPages = ['https://en.wikipedia.org/' + pages.get('href') for pages in Animals 
                       if pages.get('href') != None 
                       if pages.get('href').startswith('/wiki/')]
                       # Reduces the retrieved pages (does not work)
                       #if pages.span != None 
                       #if pages.span.attrs['class'][0] == 'tocnumber']

In [3]:
DescriptionData = {}
NonDescriptionData = {}

# Removes references in text
ReferenceRemover = '\[\d*\]'

for WikiPage in AnimalsWikiPages:
    
    # Open the page
    page = requests.get(WikiPage, timeout=5)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    for Tags in soup.find_all('h2'):
        
        # Skip useless/empty stuff
        if Tags.span == None:
            continue
        
        # Set chapter variable    
        Chapter = Tags.span.attrs['id']
        
        # Check if the chapter is description (or similar)
        if Chapter == 'Characteristics'or \
           Chapter == 'Description' or \
           Chapter == 'Appearance':
            
            
            # Get species name
            Species = soup.title\
                            .string.split(' - ')[0]\
                            .rstrip(' ')
            
            # Create list for key
            DescriptionData[Species] = []

            # Get the next sibling (text)
            for Text in Tags.find_next_siblings('p'):
                
                # But only if chapter is still valid as previous sibling
                if Chapter in Text.find_previous_siblings('h2')[0].text.strip():
                    # Add the text to the dict, remove references
                    DescriptionData[Species].append(re.sub(ReferenceRemover, '', Text.text))


In [18]:
URL = 'https://en.wikipedia.org/wiki/Brown_bear'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

DescriptionData = {}
NonDescriptionData = {}

# Removes references in text
ReferenceRemover = '\[\d*\]'

for Tags in soup.find_all('h2'):

    # Skip useless/empty stuff
    if Tags.span == None:
        continue

    # Set chapter variable    
    Chapter = Tags.span.attrs['id']

    # Check if the chapter is description (or similar)
    if Chapter == 'Characteristics'or \
       Chapter == 'Description' or \
       Chapter == 'Appearance':


        # Get species name
        Species = soup.title\
                        .string.split(' - ')[0]\
                        .rstrip(' ')

        # Create list for key
        DescriptionData[Species] = []
        NonDescriptionData[Species] = []

        # Get the next sibling (text)
        for Text in Tags.find_next_siblings('p'):

            # Add description data to dict
            if Chapter in Text.find_previous_siblings('h2')[0].text.strip():
                DescriptionData[Species].append(re.sub(ReferenceRemover, '', Text.text))
                
            # Add non description data to dict
            elif Chapter not in Text.find_previous_siblings('h2')[0].text.strip():
                 NonDescriptionData[Species].append(re.sub(ReferenceRemover, '', Text.text))

In [22]:
NonDescriptionData['Brown bear'][4]

"While the brown bear's range has shrunk and it has faced local extinctions, it remains listed as a Least concern species by the IUCN, with a total population of approximately 200,000. As of 2012[update], this and the American black bear are the only bear species not classified as threatened by the IUCN. However, the California grizzly bear, Ungavan brown bear, Atlas bear and Mexican grizzly bear, as well as brown bear populations in the Pacific Northwest, were hunted to extinction in the 19th and early 20th centuries and many of the southern Asian subspecies are highly endangered. The Syrian brown bear (Ursus arctos syriacus) is very rare and it has been extirpated from more than half of its historic range. One of the smallest-bodied subspecies, the Himalayan brown bear (Ursus arctos isabellinus), is critically endangered, occupying only 2% of its former range and threatened by uncontrolled poaching for its body parts. The Marsican brown bear in central Italy is believed to have a pop

In [5]:
df = pd.DataFrame.from_dict(textContent, orient='index')

In [12]:
df.loc['Brown bear', 1].split('. ')

['The normal range of physical dimensions for a brown bear is a head-and-body length of 1.4 to 2.8\xa0m (4\xa0ft 7\xa0in to 9\xa0ft 2\xa0in) and a shoulder height of 70 to 153\xa0cm (2\xa0ft 4\xa0in to 5\xa0ft 0\xa0in)',
 'The tail is relatively short, as in all bears, ranging from 6 to 22\xa0cm (2.4 to 8.7\xa0in) in length',
 'The smallest brown bears, females during spring among barren-ground populations, can weigh so little as to roughly match the body mass of males of the smallest living bear species, the sun bear (Helarctos malayanus), while the largest coastal populations attain sizes broadly similar to those of the largest living bear species, the polar bear',
 'Interior brown bears are generally smaller than is often perceived, being around the same weight as an average lion, at an estimate average of 180\xa0kg (400\xa0lb) in males and 135\xa0kg (298\xa0lb) in females, whereas adults of the coastal populations weigh about twice as much',
 'The average weight of adult male bears

In [ ]:
df.to_pickle("df.pkl")